# Generating pitch tracks for the Guše visualizer

In [2]:
import json
import os
import numpy as np
import os
import matplotlib.pyplot as plt

In [3]:
folderName = '../../pitchTracks/'
mbid = '67d7a5d6-e3aa-431a-a1e1-68df78c2f1a8'
fileName = mbid + '.f0_fil.csv'

dur = 58

shahed = 219.382

filePath = os.path.join(folderName, fileName)
print(filePath)

../../pitchTracks/67d7a5d6-e3aa-431a-a1e1-68df78c2f1a8.f0_fil.csv


### From Hz to cents

In [4]:
def h2c(t, h):
    return np.rint(1200 * np.log2(h/t))

In [5]:
# Load csv
pitchTrackHz = np.genfromtxt(filePath, delimiter=",")
pitchTrackHz

array([[  0.74 , 195.564],
       [  0.75 , 196.081],
       [  0.76 , 195.892],
       ...,
       [ 57.85 , 148.104],
       [ 57.86 , 148.191],
       [ 57.87 , 148.221]])

In [6]:
maxCent = h2c(shahed, np.max(pitchTrackHz[:,1]))
print("Max:", maxCent)

minCent = h2c(shahed, np.min(pitchTrackHz[:,1]))
print("Min:", minCent)

Max: 791.0
Min: -1898.0


In [21]:
minCent = -1399
minCut = True

In [22]:
# Prepare dic
pitchTrackDic = {}

for frame in pitchTrackHz:
    if minCut and frame[1] >= minCent:
        pitchTrackDic["{:.2f}".format(frame[0])] = frame[1]

In [23]:
# Find duration

if pitchTrackHz[-1,0] > dur:
    dur = pitchTrackHz[-1,0]

In [24]:
# Cent pitch track

pitchTrackCents = {}

for i in range(dur * 100):
    k = "{:.2f}".format(i/100)
    if k not in pitchTrackDic.keys():
        pitchTrackCents[k] = "s"
    else:
        pitchTrackCents[k] = h2c(shahed, pitchTrackDic[k])

pitchTrackCents

{'0.00': 's',
 '0.01': 's',
 '0.02': 's',
 '0.03': 's',
 '0.04': 's',
 '0.05': 's',
 '0.06': 's',
 '0.07': 's',
 '0.08': 's',
 '0.09': 's',
 '0.10': 's',
 '0.11': 's',
 '0.12': 's',
 '0.13': 's',
 '0.14': 's',
 '0.15': 's',
 '0.16': 's',
 '0.17': 's',
 '0.18': 's',
 '0.19': 's',
 '0.20': 's',
 '0.21': 's',
 '0.22': 's',
 '0.23': 's',
 '0.24': 's',
 '0.25': 's',
 '0.26': 's',
 '0.27': 's',
 '0.28': 's',
 '0.29': 's',
 '0.30': 's',
 '0.31': 's',
 '0.32': 's',
 '0.33': 's',
 '0.34': 's',
 '0.35': 's',
 '0.36': 's',
 '0.37': 's',
 '0.38': 's',
 '0.39': 's',
 '0.40': 's',
 '0.41': 's',
 '0.42': 's',
 '0.43': 's',
 '0.44': 's',
 '0.45': 's',
 '0.46': 's',
 '0.47': 's',
 '0.48': 's',
 '0.49': 's',
 '0.50': 's',
 '0.51': 's',
 '0.52': 's',
 '0.53': 's',
 '0.54': 's',
 '0.55': 's',
 '0.56': 's',
 '0.57': 's',
 '0.58': 's',
 '0.59': 's',
 '0.60': 's',
 '0.61': 's',
 '0.62': 's',
 '0.63': 's',
 '0.64': 's',
 '0.65': 's',
 '0.66': 's',
 '0.67': 's',
 '0.68': 's',
 '0.69': 's',
 '0.70': 's',
 '0.71

In [25]:
with open(mbid + "_pitchTrack.json", 'w') as f:
    json.dump(pitchTrackCents, f)

### Pitch histogram

In [26]:
import intonation
import numpy as np
import json
import os

In [27]:
def pitch2cents(pitch, ref):
    '''
    Converts a pitch track in Hz to cents given a reference pitch
    
    Args:
        pitch (np.array): a numpy array with two columns, first for time,
                          second for pitch
        ref (float): the reference pitch
    
    Returns:
        cents (np.array) a numpy array with one column of pitch in cents
    '''

    return 1200 * np.log2(pitch[:,1] / float(ref))

In [28]:
pitchTrackFolder = '../../pitchTracks'
mbid = '67d7a5d6-e3aa-431a-a1e1-68df78c2f1a8'
pitchTrackExtension = '.f0_fil.csv'

shahed = 219.382

In [29]:
pitchTrack = np.genfromtxt(os.path.join(pitchTrackFolder, mbid + pitchTrackExtension), delimiter=',')
centsTrack = pitch2cents(pitchTrack, shahed)
# filteredTrack = centsTrack[np.logical_and(centsTrack >= lowCut, centsTrack <= highCut)]
# pitch_obj = intonation.Pitch(np.arange(len(filteredTrack)), filteredTrack)
pitch_obj = intonation.Pitch(np.arange(len(centsTrack)), centsTrack)
rec_obj = intonation.Recording(pitch_obj)
rec_obj.compute_hist()

print(rec_obj.histogram.x)
print(rec_obj.histogram.y)

[-1897.30116697 -1896.30107462 -1895.30098227 ...   788.9468805
   789.94697285   790.9470652 ]
[0.00019177 0.00018913 0.00018393 ... 0.00032326 0.00032977 0.00033307]


In [30]:
histDic = {}

for i in range(rec_obj.histogram.x.size):
    # pos = '{:.2f}'.format(rec_obj.histogram.x[i])
    x = round(rec_obj.histogram.x[i], 2)
    y = round(rec_obj.histogram.y[i] * 10000, 2)
    histDic[x] = y

histMin = np.min(list(histDic.values()))
histMax = np.max(list(histDic.values()))

print(histMin, histMax)

histList = []
for k, v in histDic.items():
    histList.append([k,v])

hist2json = {'min': histMin, 'max': histMax, 'hist': histList}
hist2json

0.0 118.35


{'min': 0.0,
 'max': 118.35,
 'hist': [[-1897.3, 1.92],
  [-1896.3, 1.89],
  [-1895.3, 1.84],
  [-1894.3, 1.76],
  [-1893.3, 1.67],
  [-1892.3, 1.56],
  [-1891.3, 1.43],
  [-1890.3, 1.3],
  [-1889.3, 1.16],
  [-1888.3, 1.02],
  [-1887.3, 0.88],
  [-1886.3, 0.75],
  [-1885.3, 0.63],
  [-1884.3, 0.52],
  [-1883.3, 0.43],
  [-1882.3, 0.34],
  [-1881.3, 0.27],
  [-1880.3, 0.21],
  [-1879.3, 0.16],
  [-1878.3, 0.12],
  [-1877.3, 0.09],
  [-1876.3, 0.06],
  [-1875.3, 0.05],
  [-1874.3, 0.03],
  [-1873.3, 0.02],
  [-1872.3, 0.01],
  [-1871.3, 0.01],
  [-1870.3, 0.01],
  [-1869.3, 0.0],
  [-1868.3, 0.0],
  [-1867.3, 0.0],
  [-1866.3, 0.0],
  [-1865.3, 0.0],
  [-1864.3, 0.0],
  [-1863.3, 0.0],
  [-1862.3, 0.0],
  [-1861.3, 0.0],
  [-1860.3, 0.0],
  [-1859.3, 0.0],
  [-1858.3, 0.0],
  [-1857.3, 0.0],
  [-1856.3, 0.0],
  [-1855.3, 0.0],
  [-1854.3, 0.0],
  [-1853.3, 0.0],
  [-1852.3, 0.0],
  [-1851.3, 0.0],
  [-1850.3, 0.0],
  [-1849.3, 0.0],
  [-1848.3, 0.0],
  [-1847.3, 0.0],
  [-1846.3, 0.0],


In [31]:
pitchHistogramFolder = './'
pitchHistogramExtension = '_pitchHistogram.json'

json.dumps(hist2json)

with open(os.path.join(pitchHistogramFolder, mbid + pitchHistogramExtension), 'w') as f:
    json.dump(hist2json, f)